In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
# import pandas as pd

In [ ]:
!dir

In [ ]:
# # can phai split train / validate
positive_df= pd.read_csv('/kaggle/input/train-fixed/train_fixed.csv',keep_default_na=False)

In [ ]:
negative_df = pd.read_csv('/kaggle/input/negetive-train/negative_train.csv',keep_default_na=False)

In [ ]:
# print(positive_df.head(3))

In [ ]:
# print(negative_df.head(3))

In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl

In [ ]:
!pip install tensorboard

In [ ]:
import transformers
import peft
import trl

print(f"Transformers version: {transformers.__version__}")
print(f"PEFT version: {peft.__version__}")
print(f"TRL version: {trl.__version__}")

In [ ]:
import torch
import torch.nn as nn
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected!")

In [ ]:
# import torch
# import pandas as pd
from datasets import Dataset , load_from_disk
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# import numpy as np

import copy


In [ ]:
def tokenize_function(examples,tokenizer):
    tx = tokenizer(
        examples['prompt'],
        padding=False,
        truncation=True,
        max_length=3000,
        return_length=True,
        add_special_tokens=True,
    )
    return tx

In [ ]:
#xong
def preprocess_function_mini_batch(df):
    prompts = []
    for q_text, mc_answer, explanation, category, misconception in zip(
        df['QuestionText'],
        df['MC_Answer'],
        df['StudentExplanation'],
        df['Category'],
        df['Misconception']
    ):
        
        prompt = f"""<|im_start|>system
You are a meticulous educational analyst and expert in mathematics pedagogy. Your task is to perform a verification check. You will be given a student's response to a math problem, and a proposed classification for that response. You must determine if the proposed classification is entirely accurate based on the evidence.
DEFINITIONS OF THE CLASSIFICATION LABELS:
category: This is a compound label with two parts, separated by an underscore: Correctness_ReasoningType.

Part 1: Correctness (True or False): This describes whether the student's mc_answer is objectively the correct solution to the q_text.

Part 2: ReasoningType (Correct, Misconception, or Neither): This describes the quality of the student's explanation:
Correct: The explanation shows sound, logical, and mathematically valid reasoning.
Misconception: The explanation reveals a specific, identifiable error in conceptual understanding.
Neither: The explanation is incorrect, but does not point to a specific misconception. It could be a guess, irrelevant, or simply nonsensical.

misconception: This is a text description of the specific thinking error. It is only relevant when the ReasoningType in the category is Misconception. If the category is ..._Correct or ..._Neither, this field's value should be "NA" or "nan" or "NAN" ,...

YOUR STEP-BY-STEP VERIFICATION PROCESS (Chain-of-Thought):

1. Analyze Answer Correctness (True/False Check): First, independently solve the math problem in {q_text}. Compare your result to the student's {mc_answer}. Is the student's answer objectively True (correct) or False (incorrect)?
2. Analyze Explanation Quality (Reasoning Check): Now, ignore the final answer and focus only on the {explanation}.
Deconstruct the student's logic. What steps did they follow?
Based only on their text, classify their reasoning: Is it Correct, a clear Misconception, or Neither?
If you identify a misconception, briefly describe it in your own words.
3. Compare Your Analysis to the Provided Labels: Now, compare your findings from steps 1 and 2 with the given {category} and {misconception}.
Does your True/False conclusion from Step 1 match the first part of the {category} label?
Does your Correct/Misconception/Neither conclusion from Step 2 match the second part of the {category} label?
If the category is ..._Misconception, does the student's error you identified align with the provided {misconception} text?
4. Final Conclusion: A "Yes" is only possible if all checks in Step 3 pass. If there is any mismatch at any point, the answer must be "No".
Show your detailed reasoning by following these steps. Then, on the very last line, provide the final answer as exactly one word: "Yes" or "No".
<|im_end|>
<|im_start|>user
Problem Data:
Question: {q_text}
Student's Answer: {mc_answer}
Student's Explanation: {explanation}
Proposed Classification:
Category: '{category}'
Misconception: '{misconception}'
Verification Task:
Based on the data and the definitions provided, is the 'Proposed Classification' an accurate description of the 'Problem Data'?
<|im_end|>
<|im_start|>assistant
"""
        prompts.append(prompt)
    
    df['prompt'] = prompts
    # model_inputs = tokenizer(prompts, max_length=1024, truncation=True)
    # return model_inputs
    return df

In [ ]:
#chua xong
def get_dataset(df,tokenizer,save_to_disk):
    df= preprocess_function_mini_batch(df)
    dataset = Dataset.from_pandas(df)
    remove_columns = ["QuestionId","QuestionText","MC_Answer","StudentExplanation","Category","Misconception","prompt",]
    task_dataset = dataset.map(tokenize_function,batched=True,remove_columns=remove_columns,fn_kwargs = {'tokenizer' : tokenizer},batch_size=7000,num_proc=2)
    task_dataset.save_to_disk(save_to_disk)
    return load_from_disk(save_to_disk)

In [ ]:
# negative_dataset=Dataset.from_pandas(negative_df)

In [ ]:
#xong
@dataclass
class RankerDataCollator(DataCollatorWithPadding):
    tokenizer: AutoTokenizer
    
    def __call__(self, features: List[Dict[str, any]]) -> Dict[str, any]:
        batch_size = len(features)
        
        if batch_size <= 1:
            features = {k:v for k,v in feature.items() if k != 'row_id'}
            return self.tokenizer.pad(features, padding=True, return_tensors="pt")

        
        
        positive_features = list(features)
        negative_indices = []
        
        for feature in postive_features:
            idx = feature["row_id"]
            offset= idx*73
            negative_list = np.random.choice(range(offset+1,offset+73),size=batch_size,replace=False)
            negative_list = list(negative_list)
            negative_indices += negative_list
    
        
        # negative_indices = [(i + np.random.randint(1, batch_size)) % batch_size for i in range(batch_size)]
        negative_features = [negative_dataset[i] for i in negative_indices]
        
        all_features = positive_features + negative_features
        final_features = []

        for feature in all_features:
            clean = {k:v for k,v in feature.items() if k != 'row_id'}
            final_features.append(clean)

        
        padded_batch = self.tokenizer.pad(
            final_features,
            padding=True,
            return_tensors="pt",
        )
        return padded_batch

In [ ]:
decode_steps = 0

In [ ]:
#xong
class CrossEntropyTrainer(Trainer):
    def __init__(self, *args, yes_token_id, no_token_id, **kwargs):
        super().__init__(*args, **kwargs)
        self.yes_token_id = yes_token_id
        self.no_token_id = no_token_id
        self.loss_fct = nn.CrossEntropyLoss()
    
    # override
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        logits = outputs.logits

        last_token_logits = logits[:, -1, :]

        # yes_scores = last_token_logits[:, self.yes_token_id]
        # no_scores = last_token_logits[:, self.no_token_id]
        # scores = yes_scores - no_scores

        
        num_positives = self.args.per_device_train_batch_size
        num_negatives = num_positives * num_positives
        expected_total_size = num_positives + num_negatives

        if last_token_logits.shape[0] != expected_total_size:
            print(f"Batch size không khớp! Got {last_token_logits.shape[0]}, expected {expected_total_size}. Skipping loss calculation for this batch.")
            #khong hoc / update weight
            return torch.tensor(0.0, device=model.device, requires_grad=True)
        
        positive_labels = torch.full((num_positives,), self.yes_token_id, dtype=torch.long, device=model.device)
        negative_labels = torch.full((num_negatives,), self.no_token_id, dtype=torch.long, device=model.device)
        labels = torch.cat([positive_labels, negative_labels])



        if decode_steps < 4:
        
            predictions = torch.argmax(logits, dim=-1)
            
            
            decoded_inputs = self.tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=False)
            decoded_preds = self.tokenizer.decode(predictions[0], skip_special_tokens=False)
            
            print("================= DEBUG=================")
            print(f"--- INPUT ---:\n{decoded_inputs}")
            print(f"--- Response ---:\n{decoded_preds}")
            print("============================================")
            decode_steps+=1



        #ce loss
        loss = self.loss_fct(last_token_logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
MODEL_NAME= "Qwen/Qwen2.5-7B-Instruct"
# TRAIN_CSV_PATH = "/train_with_misconceptions.csv" 
OUTPUT_DIR = "./qwen2-7b-ranker-finetuned"

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)

In [ ]:
positive_dataset=get_dataset(positive_df,tokenizer)
negative_dataset=get_dataset(negative_df,tokenizer)

In [ ]:
print(positive_dataset[1])

In [ ]:
print(negative_dataset[1])

In [ ]:
def train_ranker():
    # config
    # MODEL_NAME= "Qwen/Qwen2.5-7B-Instruct"
    # # TRAIN_CSV_PATH = "/train_with_misconceptions.csv" 
    # OUTPUT_DIR = "./qwen2-7b-ranker-finetuned"

    # # load tokenizer
    # tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # if tokenizer.pad_token is None:
    #     tokenizer.pad_token = tokenizer.eos_token
        
    # # yes,no token id
    # yes_token_id = tokenizer.encode("Yes", add_special_tokens=False)[0]
    # no_token_id = tokenizer.encode("No", add_special_tokens=False)[0]
    # print(f"Token ID for 'Yes': {yes_token_id}, 'No': {no_token_id}")

    # #load dataset
    
    # # df = pd.read_csv(TRAIN_CSV_PATH)
    
    # # df = df_with_misconceptions
    
    # # raw_dataset = Dataset.from_pandas(df)
    # # tokenized_dataset = raw_dataset.map(
    # #     lambda examples: preprocess_function(examples, tokenizer),
    # #     batched=True,
    # #     remove_columns=raw_dataset.column_names # bo cot cu
    # # )

    # # tokenized_dataset = get_dataset(df,tokenizer)

    


    
    # # load model with LoRA
    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=True, bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True,
    # )
    
    # model = AutoModelForCausalLM.from_pretrained(
    #     MODEL_NAME, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
    # )

    # config PEFT(LoRA)
    model = prepare_model_for_kbit_training(model)
    peft_config = LoraConfig(
        r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    )
    model = get_peft_model(model, peft_config)
    model.config.use_cache = False # use cache

    # conf training
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=2, #batch size=2
        gradient_accumulation_steps=4, 
        learning_rate=2e-5,
        num_train_epochs=1,
        logging_steps=5,
        save_strategy="epoch",
        fp16=True,
        dataloader_num_workers=0,
        report_to="tensorboard",
    )
    
    # initialize datacollator and trainer
    data_collator = RankerDataCollator(tokenizer=tokenizer)
    
    trainer = CrossEntropyTrainer(
        model=model,
        args=training_args,
        train_dataset=positive_dataset,
        data_collator=data_collator,
        
        yes_token_id=yes_token_id,
        no_token_id=no_token_id,
    )

    # start_training
    print("Start fine-tuning với Cross-entropy Loss...\n")
    trainer.train()
    print("Training completed.\n")

    
    # save model
    final_adapter_dir = f"{OUTPUT_DIR}/final_adapters"
    trainer.save_model(final_adapter_dir)
    print(f"Adapters saved at: {final_adapter_dir}")



In [ ]:
#training
train_ranker()

In [ ]:
!zip -r qwen2_7b_ranker_finetuned.zip /kaggle/working/qwen2-7b-ranker-finetuned/final_adapters
from IPython.display import FileLink
FileLink('qwen2_7b_ranker_finetuned.zip')